In [15]:
#####設定讀取資料的年份與時段#####

year = 104    #110年或104年
rush_hour = '晨峰'    #'晨峰'或'昏峰'

inputLocation = f"{year}_現況_{rush_hour}/"

################################

import pandas as pd

df= pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]
if rush_hour == '晨峰':
    real_flow = df["晨峰實際流量"]
elif rush_hour == '昏峰':
    real_flow = df["昏峰實際流量"]
print(f'——————————————————————校估{year}年{rush_hour}指派流量——————————————————————')

real_flow = real_flow.set_index("group")


ue_results = pd.read_csv(inputLocation + f"{year}_現況_{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)


#look up UE_flow in ue_results to add a new column to sheet1 and sheet2
#for sheet2, we need to find the corresponding link name for A1, B1 and A2, B2
#that is, there are two link to be found for each row in sheet2
#then we need to add up the two link's UE_flow to get the total flow for this row
#so that we can compare with AM flow
sheet1["link_name1"] = sheet1.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
#calculate the total flow and capacity for this row

#reserve the link,AM,UE_flow
sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

group_names = sheet1['group'].unique()
groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in group_names}
for index, row in sheet1.iterrows():
    group = row['group']
    groups[group]['UE_flow_1'] += row['UE_flow_1']
    groups[group]['UE_flow_2'] += row['UE_flow_2']

# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
# {real_flow.loc[group, 'name']}
for group, flows in groups.items():
    print(f"""【{group}】 {real_flow.loc[group, 'name']}
    UE flow 往南/東 = {round(flows['UE_flow_1'])}， model diff = {(flows['UE_flow_1'] - real_flow.loc[group, '南/東']) / real_flow.loc[group, '南/東']* 100:.2f}%， CUBE diff = {(real_flow.loc[group, 'CUBE南/東'] - real_flow.loc[group, '南/東']) / real_flow.loc[group, '南/東']* 100:.2f}%
    UE flow 往北/西 = {round(flows['UE_flow_2'])}， model diff = {(flows['UE_flow_2'] - real_flow.loc[group, '北/西']) / real_flow.loc[group, '北/西']* 100:.2f}%， CUBE diff = {(real_flow.loc[group, 'CUBE北/西'] - real_flow.loc[group, '北/西']) / real_flow.loc[group, '北/西']* 100:.2f}%
    """)

——————————————————————校估104年晨峰指派流量——————————————————————
【SL1】 基隆河
    UE flow 往南/東 = 45568， model diff = 6.76%， CUBE diff = 8.69%
    UE flow 往北/西 = 45452， model diff = 7.30%， CUBE diff = -0.24%
    
【SL2】 台鐵(市民大道)
    UE flow 往南/東 = 30000， model diff = 0.60%， CUBE diff = 14.37%
    UE flow 往北/西 = 37773， model diff = -1.84%， CUBE diff = -1.63%
    
【SL3】 國1-市中心
    UE flow 往南/東 = 6207， model diff = 0.73%， CUBE diff = -17.74%
    UE flow 往北/西 = 5223， model diff = -3.26%， CUBE diff = -13.33%
    
【SL5】 臺北縣市東側運輸走廊
    UE flow 往南/東 = 8260， model diff = -8.69%， CUBE diff = -17.17%
    UE flow 往北/西 = 16891， model diff = -6.22%， CUBE diff = -19.61%
    
【SL6】 北市北側
    UE flow 往南/東 = 4949， model diff = -11.40%， CUBE diff = 1.40%
    UE flow 往北/西 = 3422， model diff = -9.42%， CUBE diff = -10.76%
    
【SL7】 臺北縣市西側運輸走廊(淡水河流域)
    UE flow 往南/東 = 32006， model diff = -19.31%， CUBE diff = 1.56%
    UE flow 往北/西 = 21075， model diff = -10.82%， CUBE diff = 9.39%
    
【SL8】 臺北縣市南側運輸走廊(新店溪&景美溪流域)
    UE f

In [11]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table

,group,link,UE_flow_1,UE_flow_2
4,SL1,中山橋,865.844829,405.583587
15,SL1,南湖大橋,2053.514937,2494.852366
5,SL1,大直橋,2477.876980,2273.367200
14,SL1,成功橋,1465.915820,1744.988681
13,SL1,成美橋,1968.376787,1380.373610
2,SL1,承德橋,1362.508867,985.858334
3,SL1,承德橋(機車道),747.987144,856.243320
7,SL1,民權大橋,1972.152477,1699.920672
8,SL1,民權大橋(機車道),1014.991984,940.954125
0,SL1,洲美快速道路,1750.281076,4615.032987
